In [1]:
import io

import netaddr

import pandas as pd

import aiohttp

from rpki_analysis.delegated_stats import read_delegated_stats, read_delegated_extended_stats

In [2]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.ripe.net/ripe/stats/nro-stats/latest/nro-delegated-stats') as resp:
        df = read_delegated_extended_stats(io.StringIO(await resp.text()))



In [3]:
df_asn = df[df.afi == 'asn'].copy()

In [4]:
from interval import interval

def interval_from_length(row) -> interval:
    return interval([row.raw_resource, int(row.raw_resource) + row.length])

In [5]:
df_asn['resource'] = df_asn.apply(interval_from_length, axis=1)

In [7]:
#df_asn[['resource', 'opaque_id'], df_asn['opaque_id'] == 'ripe'].groupby(['opaque_id']).apply(display)

In [15]:
df_asn[['opaque_id', 'rir', 'length']]\
    .groupby(['opaque_id', 'rir'])\
    .sum().reset_index()\
    .nlargest(columns=['length'], n=50)

,opaque_id,rir,length
474153,iana,iana,4199803376
474159,ietf,iana,95033875
474173,ripencc,ripencc,6197
330625,A91E5D61,apnic,4268
312787,A918EDB2,apnic,3341
310945,A9186214,apnic,3261
369806,arin,arin,2059
369799,apnic,apnic,1793
474166,lacnic,lacnic,1721
298945,A914BC7A,apnic,1654
